In [1]:
!python -V

Python 3.11.7


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [6]:
# Answers Module 1 Homework No. 1
df.shape  # Shows how many columns the raw data contains

(3066766, 19)

In [7]:
# Answers Module 1 Homework No. 2
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.duration.std()

42.594351241920904

In [8]:
# Answers Module 1 Homework No. 3
df = df[(df.duration >= 1) & (df.duration <= 60)]
df.shape  # Compare the number of rows of this output and the initial df shape

(3009173, 20)